In [11]:
import java.io.File
import com.sun.management.OperatingSystemMXBean
import java.lang.management.ManagementFactory
import kotlin.concurrent.thread
import kotlin.math.max

class ResourceMonitor {

    private val vramFile: File? by lazy {
        File("/sys/class/drm").listFiles()
            ?.filter { it.name.startsWith("card") && !it.name.contains("-") }
            ?.maxByOrNull { card ->
                File(card, "device/mem_info_vram_total").let {
                    if (it.exists()) it.readText().trim().toLongOrNull() ?: 0L else 0L
                }
            }?.let { File(it, "device/mem_info_vram_used") }
    }

    fun <T> measurePeakDelta(block: () -> T): PeakStats<T> {
        val startRam = getUsedRamGb()
        val startVram = getUsedVramGb()

        var peakRam = startRam
        var peakVram = startVram
        var running = true

        val monitorThread = thread {
            while (running) {
                peakRam = max(peakRam, getUsedRamGb())
                peakVram = max(peakVram, getUsedVramGb())
                Thread.sleep(100)
            }
        }

        val startTime = System.currentTimeMillis()
        val result = try {
            block()
        } finally {
            running = false
            monitorThread.join()
        }
        val durationSeconds = ((System.currentTimeMillis() - startTime) / 1000).toUInt()

        return PeakStats(
            result = result,
            durationSeconds = durationSeconds,
            startRamGb = startRam,
            peakRamGb = peakRam,
            startVramGb = startVram,
            peakVramGb = peakVram
        )
    }

    /**
     * Returns actual RAM used by applications (excluding buffers/cache).
     * Uses MemTotal - MemAvailable from /proc/meminfo for accurate measurement.
     */
    private fun getUsedRamGb(): Double {
        val memInfo = File("/proc/meminfo").readLines()
            .mapNotNull { line ->
                val parts = line.split(":", limit = 2)
                if (parts.size == 2) {
                    val key = parts[0].trim()
                    val value = parts[1].trim().split(" ")[0].toLongOrNull()
                    if (value != null) key to value else null
                } else null
            }
            .toMap()

        val total = memInfo["MemTotal"] ?: return 0.0
        val available = memInfo["MemAvailable"] ?: return 0.0

        // Convert from KB to GB
        return (total - available) / (1024.0 * 1024.0)
    }

    /**
     * Returns VRAM used by AMD GPU in GB.
     * Reads from /sys/class/drm/cardX/device/mem_info_vram_used
     */
    private fun getUsedVramGb(): Double {
        val bytes = vramFile?.readText()?.trim()?.toLongOrNull() ?: 0L
        return bytes / (1024.0 * 1024.0 * 1024.0)
    }

    data class PeakStats<T>(
        val result: T,
        val durationSeconds: UInt,
        val startRamGb: Double,
        val peakRamGb: Double,
        val startVramGb: Double,
        val peakVramGb: Double
    )
}

// Test cell for ResourceMonitor accuracy
fun testResourceMonitor() {
    val monitor = ResourceMonitor()

    println("=".repeat(60))
    println("Resource Monitor Accuracy Test")
    println("=".repeat(60))

    // Test RAM reading
    println("\n📊 RAM Test:")
    println("-".repeat(60))

    val memInfo = File("/proc/meminfo").readLines()
        .mapNotNull { line ->
            val parts = line.split(":", limit = 2)
            if (parts.size == 2) {
                val key = parts[0].trim()
                val value = parts[1].trim().split(" ")[0].toLongOrNull()
                if (value != null) key to value else null
            } else null
        }
        .toMap()

    val totalRamGb = (memInfo["MemTotal"] ?: 0L) / (1024.0 * 1024.0)
    val availableRamGb = (memInfo["MemAvailable"] ?: 0L) / (1024.0 * 1024.0)
    val usedRamExpected = totalRamGb - availableRamGb

    val usedRamMonitor = monitor.javaClass.getDeclaredMethod("getUsedRamGb").apply {
        isAccessible = true
    }.invoke(monitor) as Double

    println("Total RAM:              %.2f GB".format(totalRamGb))
    println("Available RAM:          %.2f GB".format(availableRamGb))
    println("Expected Used RAM:      %.2f GB".format(usedRamExpected))
    println("Monitor Reports:        %.2f GB".format(usedRamMonitor))
    println("Difference:             %.2f GB".format(kotlin.math.abs(usedRamExpected - usedRamMonitor)))
    println("Match: ${if (kotlin.math.abs(usedRamExpected - usedRamMonitor) < 0.01) "✅ PASS" else "❌ FAIL"}")

    // Test VRAM reading
    println("\n🎮 VRAM Test:")
    println("-".repeat(60))

    val vramFiles = File("/sys/class/drm").listFiles()
        ?.filter { it.name.startsWith("card") && !it.name.contains("-") }
        ?.mapNotNull { card ->
            val totalFile = File(card, "device/mem_info_vram_total")
            val usedFile = File(card, "device/mem_info_vram_used")
            if (totalFile.exists() && usedFile.exists()) {
                val total = totalFile.readText().trim().toLongOrNull() ?: 0L
                val used = usedFile.readText().trim().toLongOrNull() ?: 0L
                Triple(card.name, total, used)
            } else null
        }

    if (vramFiles.isNullOrEmpty()) {
        println("❌ No AMD GPU detected or VRAM info not available")
    } else {
        vramFiles.forEach { (cardName, total, used) ->
            val totalGb = total / (1024.0 * 1024.0 * 1024.0)
            val usedGb = used / (1024.0 * 1024.0 * 1024.0)
            println("\nCard: $cardName")
            println("  Total VRAM:           %.2f GB".format(totalGb))
            println("  Used VRAM (raw):      $used bytes")
            println("  Used VRAM:            %.2f GB".format(usedGb))
            println("  Utilization:          %.1f%%".format(usedGb / totalGb * 100))
        }

        val usedVramMonitor = monitor.javaClass.getDeclaredMethod("getUsedVramGb").apply {
            isAccessible = true
        }.invoke(monitor) as Double

        println("\nMonitor Reports:        %.2f GB".format(usedVramMonitor))
        println("Match: ${if (vramFiles.any { kotlin.math.abs(it.third / (1024.0 * 1024.0 * 1024.0) - usedVramMonitor) < 0.01 }) "✅ PASS" else "⚠️ CHECK"}")
    }

    // Test peak monitoring with a simple workload
    println("\n⏱️  Peak Monitoring Test:")
    println("-".repeat(60))

    val stats = monitor.measurePeakDelta {
        // Simulate some work
        val list = mutableListOf<ByteArray>()
        repeat(10) {
            list.add(ByteArray(10_000_000)) // Allocate ~10MB chunks
            Thread.sleep(50)
        }
        list.clear()
        "Test completed"
    }

    println("Duration:               ${stats.durationSeconds}s")
    println("Start RAM:              %.2f GB".format(stats.startRamGb))
    println("Peak RAM:               %.2f GB".format(stats.peakRamGb))
    println("RAM Delta:              %.2f GB".format(stats.peakRamGb - stats.startRamGb))
    println("Start VRAM:             %.2f GB".format(stats.startVramGb))
    println("Peak VRAM:              %.2f GB".format(stats.peakVramGb))
    println("VRAM Delta:             %.2f GB".format(stats.peakVramGb - stats.startVramGb))
    println("Result:                 ${stats.result}")

    println("\n" + "=".repeat(60))
    println("✅ Test Complete - Compare values with 'free -h' output")
    println("=".repeat(60))
}

// Run the test
testResourceMonitor()

Resource Monitor Accuracy Test

📊 RAM Test:
------------------------------------------------------------
Total RAM:              30.42 GB
Available RAM:          15.08 GB
Expected Used RAM:      15.34 GB
Monitor Reports:        15.34 GB
Difference:             0.00 GB
Match: ✅ PASS

🎮 VRAM Test:
------------------------------------------------------------

Card: card1
  Total VRAM:           15.92 GB
  Used VRAM (raw):      15873941504 bytes
  Used VRAM:            14.78 GB
  Utilization:          92.9%

Card: card0
  Total VRAM:           0.50 GB
  Used VRAM (raw):      16707584 bytes
  Used VRAM:            0.02 GB
  Utilization:          3.1%

Monitor Reports:        14.78 GB
Match: ✅ PASS

⏱️  Peak Monitoring Test:
------------------------------------------------------------
Duration:               0s
Start RAM:              15.34 GB
Peak RAM:               15.43 GB
RAM Delta:              0.09 GB
Start VRAM:             14.78 GB
Peak VRAM:              14.80 GB
VRAM Delta:        